In [ ]:
!pip install -q -r requirements.txt

In [ ]:
!pip install -q \
    --extra-index-url=https://pypi.nvidia.com \
    cudf-cu12==23.12.* dask-cudf-cu12==23.12.* cuml-cu12==23.12.* \
    cugraph-cu12==23.12.* cuspatial-cu12==23.12.* cuproj-cu12==23.12.* \
    cuxfilter-cu12==23.12.* cucim-cu12==23.12.* pylibraft-cu12==23.12.* \
    raft-dask-cu12==23.12.*

In [ ]:
from cuml import svm
from cuml import LogisticRegression
from cuml.common import logger
from additional import *
from features import *
from models import *
from models.model import model_training
import torch
import pickle as pk
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler,MinMaxScaler,FunctionTransformer, Normalizer
import numpy
import gc
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# pass 'ogbn-arxiv' to load ArXiv dataset
G, data = datasets.load_data('cora')
print(data)

# Experiment 1: basic GNN + combinations of structural and positional features

In [ ]:
# GCN setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = gcn.GCNBase(data,hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()
n_epochs = 200
n_runs = 10

In [ ]:
# global features
original_features = data.x.to(device)
structural_features = features.structural_features(G,['cc', 'bc', 'dc', 'ec', 'pr', 'cn', 'lc', 'nd', 'kc']).to(device)
positional_features = features.positional_features(data,128,50)

In [ ]:
#structural_features=utilities.load_results('structural_features')
#positional_features=utilities.load_results('positional_features')

In [ ]:
#%load_ext autoreload
#%autoreload 2

In [ ]:
def gcn_base_factory(data, hidden_channels):
  return gcn.GCNBase(data, hidden_channels)

def gcn_pre_factory(data, hidden_channels, mlp_hidden_channels):
  return gcn.GCNPre(data, hidden_channels, mlp_hidden_channels)

In [ ]:
# compute models with all feature combinations, both for base and mlp GCN
def run_feature_combinations(file_name, model_factory, model_factory_params, normalization=lambda x: x):
    features_combinations = [
      original_features,
      structural_features,
      positional_features,
      utilities.concatenate(original_features,structural_features),
      utilities.concatenate(original_features,positional_features),
      utilities.concatenate(structural_features,positional_features),
      utilities.concatenate(original_features,structural_features,positional_features)]

    file_names = [
      'original',
      'structural',
      'positional',
      'original-structural',
      'original-positional',
      'structural-positional',
      'original-structural-positional']

    basic_models = dict()
    orig_num_feat = original_features.size()[1]
    for curr_features, curr_file_name in zip(features_combinations, file_names):
        data.x = curr_features
        data.x = normalization(data.x)

        if data.name=='Cora' and (curr_file_name=='original' or curr_file_name=='original-structural' or curr_file_name=='original-positional' or curr_file_name=='original-structural-positional'):
          split = curr_features.split([orig_num_feat,curr_features.size()[1]-orig_num_feat],dim=-1)
          orig_feats = split[0]
          other_feats = split[1]
          other_feats_norm = normalization(other_feats)
          data.x = utilities.concatenate(orig_feats,other_feats_norm)

        model = model_factory(data, *model_factory_params)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
        results = dict()
        results['avg_acc'], results['test_accs'], results['train_losses'], results['train_accs'], results['val_losses'], results['val_accs'], results['run_times'],results['best_epoch'] = experiments.run_experiments(model, data, n_runs, n_epochs, optimizer, criterion, device) # These should be "global variables"

        basic_models[curr_file_name] = results

    utilities.save_results(basic_models, file_name)

In [ ]:
run_feature_combinations('gcn-base-concatenation-without-norm-cora', gcn_base_factory,[64])
gcn_base_concatenation_without_norm_cora = utilities.load_results('gcn-base-concatenation-without-norm-cora')
print(gcn_base_concatenation_without_norm_cora)

### Adding Min-Max Normalization

In [ ]:
min_max_normalization = lambda x : utilities.MinMaxNormalization(x)
run_feature_combinations('gcn-base-concatenation-minmax-norm-cora', gcn_base_factory,[64],normalization = min_max_normalization)
gcn_base_concatenation_minmax_norm_cora = utilities.load_results('gcn-base-concatenation-minmax-norm-cora')
print(gcn_base_concatenation_minmax_norm_cora)

### Adding Standard Normalization

In [ ]:
standard_normalization = lambda x : utilities.StandardNormalization(x)
run_feature_combinations('gcn-base-concatenation-standard-norm-cora', gcn_base_factory,[64],normalization = standard_normalization)
gcn_base_concatenation_standard_norm_cora = utilities.load_results('gcn-base-concatenation-standard-norm-cora')
print(gcn_base_concatenation_standard_norm_cora)

## Experiment 2: basic GCN + combinations of structural and positional feature + MLP pre-processing layer

### Basic models + 128 neurons

In [ ]:
run_feature_combinations('gcn-pre-concatenation-without-norm-cora', gcn_pre_factory,[64,128])
gcn_pre_concatenation_without_norm_cora = utilities.load_results('gcn-pre-concatenation-without-norm-cora')
print(gcn_pre_concatenation_without_norm_cora)

### Basic models + 160 neurons

In [ ]:
run_feature_combinations('gcn-pre-concatenation-without-norm-160-cora', gcn_pre_factory,[64,160])
gcn_pre_concatenation_without_norm_160_cora = utilities.load_results('gcn-pre-concatenation-without-norm-160-cora')
print(gcn_pre_concatenation_without_norm_160_cora)

### Std normalized models + 128 neurons

In [ ]:
run_feature_combinations('gcn-pre-concatenation-standard-norm-cora', gcn_pre_factory,[64,128], normalization = standard_normalization)
gcn_pre_concatenation_standard_norm_cora = utilities.load_results('gcn-pre-concatenation-standard-norm-cora')
print(gcn_pre_concatenation_standard_norm_cora)

### Std normalized models + 160 neurons

In [ ]:
run_feature_combinations('gcn-pre-concatenation-standard-norm-160-cora', gcn_pre_factory,[64,160], normalization = standard_normalization)
gcn_pre_concatenation_standard_norm_160_cora = utilities.load_results('gcn-pre-concatenation-standard-norm-160-cora')
print(gcn_pre_concatenation_standard_norm_160_cora)

## Experiment 3: basic GCN + combinations of structural and positional feature + Ensemble

In [ ]:
data_clone = data.clone()
data = data.to(device)

In [ ]:
def run_ensemble(data_orig, classifier, scaler, n_runs, file_name, normalization = lambda x: x):

  test_accs = []
  for i in range(1,n_runs+1):
    print(f"\n RUN: {i}\n")

    data = data_orig.clone()

    data.val_mask, data.ensemble_val_mask = ensemble.get_val_set_split(data)

    data.x = original_features
    if data.name!='Cora':
      data.x = normalization(data.x)
    model_original = gcn.GCNBase(data,hidden_channels=64)
    model_original = model_original.to(device)
    optimizer = torch.optim.Adam(model_original.parameters(), lr=0.01, weight_decay=5e-4)
    train_losses, train_accs, val_losses, val_accs, best_epoch = model_training(n_epochs, model_original, data, optimizer, criterion)

    print(f"\n Model with original features: training completed\n")

    data.x = positional_features
    data.x = normalization(data.x)
    model_positional = gcn.GCNBase(data,hidden_channels=64)
    model_positional = model_positional.to(device)
    optimizer = torch.optim.Adam(model_positional.parameters(), lr=0.01, weight_decay=5e-4)
    train_losses, train_accs, val_losses, val_accs, best_epoch = model_training(n_epochs, model_positional, data, optimizer, criterion)

    print(f"\n Model with positional features: training completed\n")

    models = [model_original, model_positional]
    features = [original_features, positional_features]

    meta_model_train = ensemble.get_meta_model_features(models, features, data.ensemble_val_mask, data.edge_index)
    meta_model_test = ensemble.get_meta_model_features(models, features, data.test_mask, data.edge_index)

    X_train = meta_model_train.cpu().numpy()
    y_train = data.y[data.ensemble_val_mask].cpu().numpy()
    X_test = meta_model_test.cpu().numpy()
    y_test = data.y[data.test_mask].cpu().numpy()

    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    classifier.fit(X_train_scaled, y_train)

    y_pred = classifier.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)

    test_accs.append(accuracy)

    print(f"\n Ensemble: training completed")
    print(f"Ensemble accuracy: {accuracy}")

    gc.collect()

  results = dict()
  results['test_accs'] = test_accs
  results['avg_acc'] = sum(test_accs) / len(test_accs)

  utilities.save_results(results, file_name)

## Basic models as base models

### SVM

In [ ]:
scaler = FunctionTransformer(lambda x: x)
run_ensemble(data,svm.SVC(verbose=0),scaler,5,'gcn-ensemble-SVM-without-norm-cora')
gcn_ensemble_SVM_without_norm_cora = utilities.load_results('gcn-ensemble-SVM-without-norm-cora')
print(gcn_ensemble_SVM_without_norm_cora)

### SVM + Std scaler

In [ ]:
scaler = StandardScaler()
run_ensemble(data,svm.SVC(verbose=0),scaler,5,'gcn-ensemble-SVM-standard-norm-cora')
gcn_ensemble_SVM_standard_norm_cora = utilities.load_results('gcn-ensemble-SVM-standard-norm-cora')
print(gcn_ensemble_SVM_standard_norm_cora)

### Logistic Regressor

In [ ]:
scaler = FunctionTransformer(lambda x: x)
run_ensemble(data,LogisticRegression(max_iter=10000, multi_class="multinomial",verbose=0),scaler,5,'gcn-ensemble-LR-without-norm-cora')
gcn_ensemble_LR_without_norm_cora = utilities.load_results('gcn-ensemble-LR-without-norm-cora')
print(gcn_ensemble_LR_without_norm_cora)

### Logistic Regressor + Std

In [ ]:
scaler = StandardScaler()
run_ensemble(data,LogisticRegression(max_iter=10000, multi_class="multinomial",verbose=0),scaler,5,'gcn-ensemble-LR-standard-norm-cora')
gcn_ensemble_LR_standard_norm_cora = utilities.load_results('gcn-ensemble-LR-standard-norm-cora')
print(gcn_ensemble_LR_standard_norm_cora)

### Decision Tree

In [ ]:
scaler = FunctionTransformer(lambda x: x)
run_ensemble(data,DecisionTreeClassifier(),scaler,5,'gcn-ensemble-DT-without-norm-cora')
gcn_ensemble_DT_without_norm_cora = utilities.load_results('gcn-ensemble-DT-without-norm-cora')
print(gcn_ensemble_DT_without_norm_cora)

### Decision Tree + Std

In [ ]:
scaler = StandardScaler()
run_ensemble(data,DecisionTreeClassifier(),scaler,5,'gcn-ensemble-DT-standard-norm-cora')
gcn_ensemble_DT_standard_norm_cora = utilities.load_results('gcn-ensemble-DT-standard-norm-cora')
print(gcn_ensemble_DT_standard_norm_cora)